In [1]:
import pandas as pd
import numpy as np
import pickle

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.layers import Input, Dense
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [2]:
df_train=pd.read_csv('train.csv',index_col=False)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
1,177,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2,0,0,132,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,10
3,255,255,255,255,255,255,255,255,214,0,...,0,0,0,0,0,0,0,0,0,10
4,255,255,255,255,255,255,255,114,0,0,...,0,0,0,0,0,0,0,0,0,10


In [3]:
labels=df_train[['784']]

In [4]:
df_train.drop(df_train.columns[[784]],axis=1,inplace=True)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,177,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,132,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
3,255,255,255,255,255,255,255,255,214,0,...,0,0,0,0,0,0,0,0,0,0
4,255,255,255,255,255,255,255,114,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
np.random.seed(1212)

In [6]:
labels=np.array(labels)

In [7]:
from keras.utils.np_utils import to_categorical
cat=to_categorical(labels,num_classes=17)

In [8]:
print(cat[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [9]:
l=[]
for i in range(10548):
    l.append(np.array(df_train[i:i+1]).reshape(28,28,1))

In [10]:
np.random.seed(7)

In [11]:
model=Sequential()
model.add(Conv2D(30, (5, 5), input_shape=( 28, 28,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(206, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(17, activation='softmax'))

In [12]:
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

In [13]:
model.fit(np.array(l), cat, epochs=10, shuffle=True)

Epoch 1/10
330/330 [==============================] - 6s 15ms/step - loss: 1.3530 - accuracy: 0.6083
Epoch 2/10
330/330 [==============================] - 5s 15ms/step - loss: 0.4897 - accuracy: 0.8366
Epoch 3/10
330/330 [==============================] - 5s 14ms/step - loss: 0.3423 - accuracy: 0.8802
Epoch 4/10
330/330 [==============================] - 5s 14ms/step - loss: 0.2764 - accuracy: 0.9041
Epoch 5/10
330/330 [==============================] - 5s 14ms/step - loss: 0.2186 - accuracy: 0.9175
Epoch 6/10
330/330 [==============================] - 5s 14ms/step - loss: 0.1961 - accuracy: 0.9280
Epoch 7/10
330/330 [==============================] - 5s 14ms/step - loss: 0.1875 - accuracy: 0.9303
Epoch 8/10
330/330 [==============================] - 5s 14ms/step - loss: 0.1704 - accuracy: 0.9327
Epoch 9/10
330/330 [==============================] - 5s 15ms/step - loss: 0.1564 - accuracy: 0.9405
Epoch 10/10
330/330 [==============================] - 5s 14ms/step - loss: 0.1338 - accura

In [20]:
model.save('model.keras')

In [21]:
import keras
modelI=keras.models.load_model('model.keras')

In [30]:
import cv2

img=cv2.imread("test/test63.jpg",cv2.IMREAD_GRAYSCALE)
cv2.imshow("Image",img)
cv2.waitKey(0)
cv2.destroyAllWindows()
if img is not None:
    img=~img
    ret,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    ctrs,ret=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnt=sorted(ctrs,key=lambda ctr:cv2.boundingRect(ctr)[0])
    w=int(28)
    h=int(28)
    train_data=[]
    rects=[]
    for c in cnt:
        x,y,w,h=cv2.boundingRect(c)
        rect=[x,y,w,h]
        rects.append(rect)
#     print("rects", rects)
    bool_rect=[]
    for r in rects:
        l=[]
        for rec in rects:
            flag=0
            if rec!=r:
                if r[0]<(rec[0]+rec[2]+10) and rec[0]<(r[0]+r[2]+10) and r[1]<(rec[1]+rec[3]+10) and rec[1]<(r[1]+r[3]+10):
                    flag=1
                l.append(flag)
            if rec==r:
                l.append(0)
        bool_rect.append(l)
#     print("bools",bool_rect)
    dump_rect=[]
    for i in range(0,len(cnt)):
        for j in range(0,len(cnt)):
            if bool_rect[i][j]==1:
                area1=rects[i][2]*rects[i][3]
                area2=rects[j][2]*rects[j][3]
                if(area1==min(area1,area2)):
                    dump_rect.append(rects[i])
#     print("dump_rects",dump_rect)
    final_rect=[i for i in rects if i not in dump_rect]
#     print("Final_rects",final_rect)
    for r in final_rect:
        x=r[0]
        y=r[1]
        w=r[2]
        h=r[3]
        im_crop=thresh[y:y+h+10,x:x+w+10]
        im_resize=cv2.resize(im_crop,(28,28))
#         cv2.imshow("work",im_resize)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        train_data.append(im_resize)

In [31]:
import numpy as np
s=""
for i in range(len(train_data)):
    train_data[i]=np.array(train_data[i])
    train_data[i]=train_data[i].reshape(1,28,28,1)
#     result=modelI.predictclasses(train_data[i])
    result=np.argmax(model.predict(train_data[i]), axis=1)
    if(result[0]==10):
        s=s+'-'
    if(result[0]==11):
        s=s+'+'
    if(result[0]==0):
        s=s+'0'
    if(result[0]==1):
        s=s+'1'
    if(result[0]==2):
        s=s+'2'
    if(result[0]==3):
        s=s+'3'
    if(result[0]==4):
        s=s+'4'
    if(result[0]==5):
        s=s+'5'
    if(result[0]==6):
        s=s+'6'
    if(result[0]==7):
        s=s+'7'
    if(result[0]==8):
        s=s+'8'
    if(result[0]==9):
        s=s+'9'
    if(result[0]==12):
        s=s+'*'
    if(result[0]==13):
        s=s+'='
    if(result[0]==14):
        s=s+'a'
    if(result[0]==15):
        s=s+'b'
    if(result[0]==16):
        s=s+'c'
print(s)

a-b-5


In [32]:
# eval(s)

NameError: name 'a' is not defined